In [ ]:
import joblib

model = joblib.load("/Users/borismartinez/Documents/GitHub/engage/model.pkl")
print(type(model))
print(model)

FileNotFoundError: [Errno 2] No such file or directory: 'model.pkl'

In [11]:
# loading Data
import pandas as pd
from matching.loaders.load_attempts import filter_dataframe_by_columns


matches = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_match_export.csv")

attempts = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_blocks_export.csv", low_memory=False)

# Specify columns that must NOT contain NA values
required_columns = ["first_name", "last_name"]
attempts = filter_dataframe_by_columns(attempts, required_columns)
attempts = attempts[attempts["upload_time"].astype(str).str.contains("2025", na=False)]

 
#
pledges = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/pledge_data.csv")


In [8]:
import pandas as pd
from matching.loaders.load_attempts import filter_dataframe_by_columns

# Load CSV
attempts = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_blocks_export.csv", low_memory=False)

# Filter to rows having 'upload_time' containing '2025'
attempts_2025 = attempts
print(f"Total rows with upload_time containing '2025': {len(attempts_2025)}")

# Boolean masks for empty or null first_name or last_name
empty_first_name = attempts_2025['first_name'].astype(str).str.strip().eq('')
empty_last_name = attempts_2025['last_name'].astype(str).str.strip().eq('')

missing_either_count = attempts_2025[empty_first_name | empty_last_name].shape[0]

print(f"Rows with empty or missing first_name OR last_name: {missing_either_count}")

# Optionally, count rows with non-empty both
required_columns = ["first_name", "last_name"]
valid_attempts = filter_dataframe_by_columns(attempts_2025, required_columns)
print(f"Rows with non-empty first_name AND last_name: {len(valid_attempts)}")

Total rows with upload_time containing '2025': 84616
Rows with empty or missing first_name OR last_name: 0
Rows with non-empty first_name AND last_name: 51008


In [10]:
import pandas as pd

# Load your dataset
attempts = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_blocks_export.csv", low_memory=False)

# Filter rows with 'upload_time' containing '2025'
attempts_2025 = attempts

# Define a function to identify missing or null-like strings
def is_missing(series):
    cleaned = series.astype(str).str.strip()
    cleaned_lower = cleaned.str.lower()
    null_values = ['', 'na', 'nan', 'null', 'none', 'undefined']
    return cleaned_lower.isin(null_values)

# Filter rows where BOTH first_name and last_name are missing/null
null_first_last = attempts_2025[is_missing(attempts_2025['first_name']) & is_missing(attempts_2025['last_name'])]

print(f"Number of rows with null/empty first and last name: {len(null_first_last)}")
print(null_first_last.head(10))

Number of rows with null/empty first and last name: 31847
                    upload_time voter_registration_scan_updated_at  \
7    2018-10-07 01:10:08.114161                2025-01-28 05:44:49   
17    2018-10-05 00:03:56.58138                2025-01-28 05:44:37   
20   2018-10-07 01:10:52.357695                2025-01-28 05:44:37   
26          2025-08-20 18:10:09                2025-08-20 18:10:09   
52   2018-10-07 01:10:52.912694                2025-01-28 05:44:37   
85   2018-10-07 01:04:30.539477                2025-01-28 05:45:06   
95   2018-10-07 01:05:22.399905                2025-01-28 05:45:03   
96          2025-08-20 18:35:59                2025-08-20 18:35:59   
131         2025-08-20 18:35:53                2025-08-20 18:35:53   
135  2018-10-07 01:08:35.851988                2025-01-28 05:44:53   

    visual_qc_county is_registration_form  registration_form_id  \
7                NaN                    f            6000338594   
17               NaN                 

In [7]:
distinct_org_packet = null_first_last[['organization_name', 'packet_filename']].drop_duplicates()
print(f"Distinct organization_name and packet_filename combinations where both first and last name are null:\n")
print(distinct_org_packet)

Distinct organization_name and packet_filename combinations where both first and last name are null:

                    organization_name  \
26                     Engage - Miami   
96                     Engage - Miami   
131    Florida Civic Engagement Table   
203    Florida Civic Engagement Table   
205                    Engage - Miami   
...                               ...   
84572                  Engage - Miami   
84574                  Engage - Miami   
84589                  Engage - Miami   
84599                  Engage - Miami   
84614                  Engage - Miami   

                                      packet_filename  
26                 Engage_Miami_GP49_4.10.2018_AM.pdf  
96                 Engage_Miami_GP49_4.12.2018_AM.pdf  
131                Engage_Miami_FT96_4.18.2018_AM.pdf  
203                Engage_Miami_NM45_4.29.2018_PM.pdf  
205                Engage_Miami_MF28_4.29.2018_PM.pdf  
...                                               ...  
84572        

In [5]:
null_equivs = ['', 'na', 'nan', 'null', 'none', 'undefined']

for val in null_equivs:
    count_fn = (attempts_2025['first_name'].astype(str).str.lower().str.strip() == val).sum()
    count_ln = (attempts_2025['last_name'].astype(str).str.lower().str.strip() == val).sum()
    print(f"First name '{val}': {count_fn}, Last name '{val}': {count_ln}")

First name '': 0, Last name '': 0
First name 'na': 0, Last name 'na': 0
First name 'nan': 30675, Last name 'nan': 30670
First name 'null': 0, Last name 'null': 0
First name 'none': 0, Last name 'none': 0
First name 'undefined': 0, Last name 'undefined': 0


In [4]:
import numpy as np

bins = [0, 0.7, 0.8, 0.9, 0.95, 0.99, 1.00001]
labels = ["<=0.70", "0.70–0.80", "0.80–0.90", "0.90–0.95", "0.95–0.99", "0.99–1.00"]

matches["conf_bin"] = pd.cut(matches["confidence_score"], bins=bins, labels=labels, include_lowest=True)

print(matches["conf_bin"].value_counts(dropna=False))

ct = pd.crosstab(matches["type_code"], matches["conf_bin"], normalize="index").round(3)
ct

conf_bin
0.99–1.00    3859
0.95–0.99    1105
0.90–0.95     578
0.80–0.90     273
0.70–0.80      80
<=0.70          4
NaN             2
Name: count, dtype: int64


conf_bin,<=0.70,0.70–0.80,0.80–0.90,0.90–0.95,0.95–0.99,0.99–1.00
type_code,,,,,,
Cross-state move,0.000,0.000,0.077,0.169,0.169,0.585
Date change,0.000,0.000,0.025,0.275,0.225,0.475
First time registrant,0.001,0.006,0.020,0.057,0.175,0.741
In-state move cross-county,0.000,0.004,0.030,0.143,0.217,0.604
In-state move in-county,0.000,0.003,0.070,0.118,0.269,0.541
No change registered,0.000,0.006,0.028,0.084,0.270,0.613
No change unregistered,0.000,0.182,0.455,0.273,0.091,0.000
No match,0.001,0.016,0.072,0.144,0.121,0.647
Other,0.000,0.000,0.308,0.385,0.077,0.231


In [5]:
summary = (
    matches
    .groupby("type_code")["confidence_score"]
    .describe()
    .sort_values("mean", ascending=False)
)
summary

,count,mean,std,min,25%,50%,75%,max
type_code,,,,,,,,
First time registrant,2807.0,0.985875,0.031509,0.687259,0.984375,1.000000,1.000000,1.000000
Status change,123.0,0.980649,0.039313,0.727495,0.964830,1.000000,1.000000,1.000000
No change registered,538.0,0.979416,0.033269,0.780921,0.964143,0.992248,1.000000,1.000000
In-state move cross-county,230.0,0.977076,0.034559,0.780921,0.964143,0.996124,1.000000,1.000000
Other state registered,27.0,0.973984,0.061231,0.707692,0.980260,1.000000,1.000000,1.000000
Cross-state move,65.0,0.973645,0.037102,0.882353,0.955823,1.000000,1.000000,1.000000
In-state move in-county,603.0,0.972371,0.039328,0.752852,0.964143,0.992248,1.000000,1.000000
No match,1146.0,0.971927,0.048635,0.678495,0.964143,1.000000,1.000000,1.000000
Date change,40.0,0.966602,0.038635,0.864925,0.925620,0.985521,1.000000,1.000000


In [6]:
att = attempts.merge(matches[["registration_form_id", "type_code", "confidence_score"]],
                     on="registration_form_id", how="inner")

In [ ]:
import matching.utils.db as db

# test connection: just count rows
df_count = db.run_query("SELECT COUNT(*) AS n FROM voterfile.election_detail_2018;")
print(df_count)

# load a specific snapshot (be careful, these tables can be huge)
vf_2018 = db.run_query("SELECT * FROM voterfile.election_detail_2018 where county = 'DAD';")
vf_2018.head()

          n
0  14176435


,county,voter_id,suffix_name,last_name,first_name,middle_name,requested_public_record,residence_address_1,residence_address_2,residence_city_usps,...,daytime_phone_extension,former_name,voting_assistance,poll_worker,state_country_of_birth,military,military_dependent,overseas,vote_history_code,email
0,DAD,110190324,None,Garcia,Zoraida,None,N,130 NW 24Th CT,,Miami,...,None,None,N,N,cuba,N,N,N,N,None
1,DAD,109372330,None,Fagundo,Gisell,None,N,6390 SW 30Th St,,Miami,...,None,None,N,N,CU,N,N,N,N,None
2,DAD,109827417,None,Torres,Isabel,None,N,720 W 75Th St,,Hialeah,...,None,None,N,N,None,N,N,N,N,None
3,DAD,109827418,None,Martinez,Maria,Elena,N,8302 SW 148Th Ave,,Miami,...,None,None,N,N,None,N,N,N,N,None
4,DAD,109827419,None,Dubuisson,Emilienne,None,N,1042 NW 110Th St,,Miami,...,None,None,Y,N,None,N,N,N,N,None


In [6]:
vf_cols_relevant = [
    # Identity
    "voter_id",
    "first_name",
    "middle_name",
    "last_name",
    "suffix_name",
    
    # Core address (for matching to voting_street_address)
    "residence_address_1",
    "residence_address_2",
    "residence_city_usps",
    "residence_state",
    "residence_zipcode",
    
    # Mailing address (secondary signal)
    "mailing_address_1",
    "mailing_address_2",
    "mailing_city",
    "mailing_state",
    "mailing_zipcode",
    
    # Demographics / registration info
    "gender",
    "race",
    "birth_date",
    "reg_date",
    "party",
    
    # Geography / district info
    "county",
    "precinct",
    "precinct_group",
    "precinct_split",
    "precinct_suffix",
    "congressional_district",
    "house_district",
    "senate_district",
    "county_commission_district",
    "school_board_district",
    
    # Contact (weaker, but still useful)
    "daytime_area_code",
    "daytime_phone_num",
    "email",
    
    # Status flags
    "voter_status",
]

att_cols_relevant = [
    # Labels from match result
    "type_code",
    "confidence_score",
    
    # Identity
    "first_name",
    "middle_name",
    "last_name",
    "name_suffix",
    "name_prefix",
    "registration_form_id",

    
    # Core voting address (to match to residence_address_*)
    "voting_street_address_one",
    "voting_street_address_two",
    "voting_city",
    "voting_state",
    "voting_zipcode",
    
    # Mailing address (secondary signal)
    "mailing_street_address_one",
    "mailing_street_address_two",
    "mailing_city",
    "mailing_zipcode",
    
    # Demographics / registration info
    "gender",
    "date_of_birth",
    "party",
    "ethnicity",          # may map loosely to race
    
    # Contact
    "phone_number",
    "email_address",
    
    # Geospatial hints about where they were registered
    "data_entry_county",
    "program_state",
    "collection_location_county",
    "collection_location_city",
    "collection_location_zip",
    "voting_address_latitude",
    "voting_address_longitude",
    
    # Quality / address flags
    "address_validated",
    "eligible_voting_age",
]

In [7]:
vf_small  = vf_2018[vf_cols_relevant].copy()
att_small = att[att_cols_relevant].copy()

In [ ]:
att_small.rename(columns={
    "first_name": "first_name_att",
    "last_name": "last_name_att",
    "voting_zipcode": "zip_raw_att",
    "date_of_birth": "dob_raw_att",
}, inplace=True)

vf_small.rename(columns={
    "first_name": "first_name_vf",
    "last_name": "last_name_vf",
    "residence_zipcode": "zip_raw_vf",
    "birth_date": "dob_raw_vf",
}, inplace=True)

In [9]:
import pandas as pd
import numpy as np
import re
import unicodedata

def normalize_text(s):
    if pd.isna(s):
        return None
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s]", "", s)
    s = re.sub(r"\s+", " ", s)
    return s or None

def normalize_zip(z):
    if pd.isna(z):
        return None
    z = str(z).strip()
    z = re.sub(r"\D", "", z)      # keep only digits
    return z[:5] or None          # use 5-digit ZIP

def to_date(col):
    return pd.to_datetime(col, errors="coerce").dt.date

In [10]:
att_small["fn_norm_att"] = att_small["first_name_att"].apply(normalize_text)
att_small["ln_norm_att"] = att_small["last_name_att"].apply(normalize_text)
att_small["zip_norm_att"] = att_small["zip_raw_att"].apply(normalize_zip)
att_small["dob_norm_att"] = pd.to_datetime(att_small["dob_raw_att"], errors="coerce")

att_small["dob_year_att"] = att_small["dob_norm_att"].dt.year.astype("Int64")

In [ ]:
vf_small["fn_norm_vf"] = vf_small["first_name_vf"].apply(normalize_text)
vf_small["ln_norm_vf"] = vf_small["last_name_vf"].apply(normalize_text)
vf_small["zip_norm_vf"] = vf_small["zip_raw_vf"].apply(normalize_zip)

# FIRST convert DOB
vf_small["dob_norm_vf"] = pd.to_datetime(vf_small["dob_raw_vf"], errors="coerce")

# THEN extract the year
vf_small["dob_year_vf"] = vf_small["dob_norm_vf"].dt.year.astype("Int64")

In [13]:
clean_pairs = att_small.merge(
    vf_small,
    left_on=["fn_norm_att", "ln_norm_att", "dob_norm_att", "zip_norm_att"],
    right_on=["fn_norm_vf", "ln_norm_vf", "dob_norm_vf", "zip_norm_vf"],
    how="inner",
    suffixes=("_att", "_vf")
)

pos_df = clean_pairs[[
    "registration_form_id",
    "first_name_att", "last_name_att", "dob_norm_att", "zip_norm_att",
    "first_name_vf",  "last_name_vf",  "dob_norm_vf",  "zip_norm_vf"
]].copy()

pos_df["is_match"] = 1

Build a strict deterministic join to find “gold” matches

We want to throw away ambiguous cases (multiple voterfile rows for the same attempt).


In [16]:
# Count unique voter_ids matched to each attempt
attempt_match_counts = (
    clean_pairs.groupby("registration_form_id")["voter_id"]
         .nunique()
         .reset_index(name="n_voters")
)

# Keep attempts with EXACTLY 1 match
good_attempts = attempt_match_counts[attempt_match_counts["n_voters"] == 1]["registration_form_id"]

gold_pairs = clean_pairs[clean_pairs["registration_form_id"].isin(good_attempts)].copy()
len(gold_pairs)

3081

In [17]:
gold_pairs["type_code"].value_counts()

type_code
First time registrant         1869
In-state move in-county        431
No change registered           387
In-state move cross-county     135
Status change                   82
Other address registered        77
Cross-state move                42
Date change                     28
Other state registered          14
No match                        12
Other                            2
Other address unregistered       1
No change unregistered           1
Name: count, dtype: int64

In [18]:
gold_pairs.groupby("type_code")["confidence_score"].describe().sort_values("mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
type_code,,,,,,,,
Other address unregistered,1.0,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
No match,10.0,0.999225,0.002451,0.992248,1.000000,1.000000,1.000000,1.000000
Other state registered,14.0,0.994913,0.013144,0.951923,1.000000,1.000000,1.000000,1.000000
First time registrant,1869.0,0.990639,0.025924,0.687259,0.992248,1.000000,1.000000,1.000000
Status change,82.0,0.986450,0.032526,0.757048,0.986343,1.000000,1.000000,1.000000
No change registered,387.0,0.986179,0.026684,0.785714,0.984375,1.000000,1.000000,1.000000
In-state move cross-county,135.0,0.982513,0.030285,0.884120,0.965713,1.000000,1.000000,1.000000
Cross-state move,42.0,0.980694,0.034017,0.882353,0.975904,1.000000,1.000000,1.000000
In-state move in-county,431.0,0.976521,0.038726,0.775254,0.964143,0.992248,1.000000,1.000000


In [19]:
# You can peek at some rows:
cols_to_view = [
    "first_name_att", "last_name_att", "date_of_birth",
    "voting_street_address_one", "voting_city", "voting_zipcode",
    "first_name_vf", "last_name_vf", "birth_date",
    "residence_address_1", "residence_city_usps", "residence_zipcode",
    "type_code", "confidence_score"
]

available_cols = [c for c in cols_to_view if c in gold_pairs.columns]

gold_pairs[available_cols].head(20)


,first_name_att,last_name_att,voting_street_address_one,voting_city,first_name_vf,last_name_vf,residence_address_1,residence_city_usps,type_code,confidence_score
0,Michael,Gonzalez,16271 SW 77th Ter,Miami,Michael,Gonzalez,16271 SW 77Th TER,Miami,No change registered,1.000000
1,Kimberleann,Montinat,1750 NW 93rd Ter,Miami,Kimberleann,Montinat,1750 NW 93Rd TER,Miami,First time registrant,1.000000
2,Benedicte,Manasse,17415 NE 11th Ave,Miami,Benedicte,Manasse,17415 NE 11Th AVE,Miami,No change registered,1.000000
3,Javier,Mercado,8202 NW 191st Ln,Hialeah,Javier,Mercado,8202 NW 191St LN,Hialeah,No change registered,1.000000
4,Withny,Cedor,18525 ww 22 ct,Miami Gardens,Withny,Cedor,18525 NW 22Nd CT,Miami Gardens,First time registrant,0.930769
5,Haley,Rodriguez,6360 mw 114th Ave,doral,Haley,Rodriguez,6360 NW 114Th AVE,Doral,In-state move in-county,0.984615
6,Joseph,Johnson,18900 NW 14th Ct,Miami,Joseph,Johnson,18900 NW 14Th CT,Miami Gardens,No change registered,0.964143
7,Oyeniyi,Amoda,3262 NW 203rd St,Miami Gardens,Oyeniyi,Amoda,3262 NW 203Rd ST,Miami Gardens,In-state move in-county,1.000000
8,Michael,Vives,650 se 6nd place,HIALEAH,Michael,Vives,650 SE 2Nd PL,Hialeah,In-state move cross-county,0.930769
9,Morgan,Lewis,1250 Alton Rd # 40,Miami Beach,Morgan,Lewis,1250 Alton RD,Miami Beach,Cross-state move,0.882353


In [20]:
match_types = [
    "First time registrant",
    "Status change",
    "In-state move cross-county",
    "In-state move in-county",
    "Cross-state move",
    # add others you consider "true matches"
]

gold_pairs["is_match"] = gold_pairs["type_code"].isin(match_types).astype(int)
gold_pairs["is_match"].value_counts()

is_match
1    2559
0     522
Name: count, dtype: int64

Create candidate pairs (blocking)

In [21]:
candidates = att_small.merge(
    vf_small,
    left_on=["ln_norm_att", "dob_year_att"],
    right_on=["ln_norm_vf", "dob_year_vf"],
    how="inner",
    suffixes=("_att", "_vf")
)

 Build similarity features

In [22]:
from rapidfuzz.distance import JaroWinkler

In [24]:
from rapidfuzz.distance import JaroWinkler

def compute_similarity(row):
    row["fn_jw"] = JaroWinkler.normalized_similarity(
        row["first_name_att"] or "",
        row["first_name_vf"] or ""
    )
    row["ln_jw"] = JaroWinkler.normalized_similarity(
        row["last_name_att"] or "",
        row["last_name_vf"] or ""
    )

    row["dob_exact"] = int(row["dob_norm_att"] == row["dob_norm_vf"])
    row["zip_exact"] = int(row["zip_norm_att"] == row["zip_norm_vf"])
    return row

Generate Synthetic Negatives

In [25]:
import random
from collections import defaultdict

# ------------------------------------------------------
# HELPER: clean extraction of first letter safely
# ------------------------------------------------------
def safe_first_letter(x):
    if pd.isna(x):
        return None
    x = str(x).strip()
    return x[0].lower() if len(x) > 0 else None


# ------------------------------------------------------
# LIMIT VF TO MIAMI-DADE FOR NEGATIVE SAMPLING
# ------------------------------------------------------
vf_miami = vf_small[vf_small["county"] == "DAD"].copy()

print("Miami-Dade voterfile size:", len(vf_miami))


# ------------------------------------------------------
# BUILD FAST LOOKUP INDEXES
# ------------------------------------------------------

zip_index = defaultdict(list)     # key: 5-digit ZIP -> list of voters
dob_index = defaultdict(list)     # key: dob_norm_vf -> list of voters
name_index = defaultdict(list)    # key: (firstletter, lastletter) -> list of voters

for _, r in vf_miami.iterrows():
    # ZIP
    zip_index[r["zip_norm_vf"]].append(r)

    # DOB
    dob_index[r["dob_norm_vf"]].append(r)

    # Name first letters
    fn0 = safe_first_letter(r["fn_norm_vf"])
    ln0 = safe_first_letter(r["ln_norm_vf"])
    if fn0 and ln0:
        name_index[(fn0, ln0)].append(r)


print("Indexing complete.")


# ------------------------------------------------------
# GENERATE NEGATIVE SAMPLES
# ------------------------------------------------------

negatives = []

for _, row in pos_df.iterrows():

    # ---------------------------
    # A: Same ZIP, different DOB
    # ---------------------------
    zip_list = zip_index.get(row["zip_norm_att"], [])
    zip_matches = [
        r for r in zip_list
        if r["dob_norm_vf"] != row["dob_norm_att"]
    ]

    # ---------------------------
    # B: Same DOB, different last name
    # ---------------------------
    dob_list = dob_index.get(row["dob_norm_att"], [])
    dob_matches = [
        r for r in dob_list
        if r["ln_norm_vf"] != row["last_name_att"]
    ]

    # ---------------------------
    # C: Similar name start letters
    # ---------------------------
    fn0 = safe_first_letter(row["first_name_att"])
    ln0 = safe_first_letter(row["last_name_att"])

    name_matches = (
        name_index.get((fn0, ln0), [])
        if fn0 and ln0 else
        []
    )

    candidate_groups = [zip_matches, dob_matches, name_matches]

    # add negatives from each group
    for group in candidate_groups:
        if group:
            wrong = random.choice(group)

            negatives.append({
                "registration_form_id": row["registration_form_id"],

                "first_name_att": row["first_name_att"],
                "last_name_att":  row["last_name_att"],
                "dob_norm_att":   row["dob_norm_att"],
                "zip_norm_att":   row["zip_norm_att"],

                "first_name_vf": wrong["first_name_vf"],
                "last_name_vf":  wrong["last_name_vf"],
                "dob_norm_vf":   wrong["dob_norm_vf"],
                "zip_norm_vf":   wrong["zip_norm_vf"],

                "is_match": 0
            })

    # ------------------------------------------------
    # ALWAYS include 1 pure random negative
    # ------------------------------------------------
    wrong = vf_miami.sample(1).iloc[0]
    negatives.append({
        "registration_form_id": row["registration_form_id"],

        "first_name_att": row["first_name_att"],
        "last_name_att":  row["last_name_att"],
        "dob_norm_att":   row["dob_norm_att"],
        "zip_norm_att":   row["zip_norm_att"],

        "first_name_vf": wrong["first_name_vf"],
        "last_name_vf":  wrong["last_name_vf"],
        "dob_norm_vf":   wrong["dob_norm_vf"],
        "zip_norm_vf":   wrong["zip_norm_vf"],

        "is_match": 0
    })

print("Negatives generated:", len(negatives))

Miami-Dade voterfile size: 1500082
Indexing complete.
Negatives generated: 12324


In [27]:
# Combine positives + negatives
train_df = pd.concat(
    [pos_df, pd.DataFrame(negatives)],
    ignore_index=True
)

# Compute similarity features
train_df = train_df.apply(compute_similarity, axis=1)

# Convert booleans
train_df["dob_exact"] = train_df["dob_exact"].astype(int)
train_df["zip_exact"] = train_df["zip_exact"].astype(int)

# Feature columns
feature_cols = [
    "fn_jw",
    "ln_jw",
    "dob_exact",
    "zip_exact",
]

X = train_df[feature_cols]
y = train_df["is_match"]


In [28]:
# Train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

# Train model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

# Evaluate
from sklearn.metrics import classification_report, roc_auc_score
preds = model.predict(X_test)
probs = model.predict_proba(X_test)[:,1]

print(classification_report(y_test, preds))
print("AUC:", roc_auc_score(y_test, probs))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3082
           1       0.97      1.00      0.99       770

    accuracy                           0.99      3852
   macro avg       0.99      1.00      0.99      3852
weighted avg       0.99      0.99      0.99      3852

AUC: 0.9968531144390975


In [29]:
# Define the columns used as model features
feature_cols = [
    "fn_jw",
    "ln_jw",
    "dob_exact",
    "zip_exact",
]

# Feature matrix and target
X = train_df[feature_cols]
y = train_df["is_match"]

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

In [31]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,2000
,multi_class,'deprecated'


In [32]:
from sklearn.metrics import classification_report, roc_auc_score

preds = model.predict(X_test)
probs = model.predict_proba(X_test)[:,1]

print(classification_report(y_test, preds))
print("AUC:", roc_auc_score(y_test, probs))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3082
           1       0.97      1.00      0.99       770

    accuracy                           0.99      3852
   macro avg       0.99      1.00      0.99      3852
weighted avg       0.99      0.99      0.99      3852

AUC: 0.9968531144390975


In [33]:
# After your model evaluation in the prototype...

print("\n=== POSITIVE PAIRS DATA ===")
print(f"Columns: {pos_df.columns.tolist()}")
print(f"Shape: {pos_df.shape}")
print(pos_df["is_match"].value_counts())

print("\n=== TRAINING DATA ===")
print(f"Shape: {train_df.shape}")
print(train_df["is_match"].value_counts())

print("\n=== FEATURES and TARGET ===")
print(f"Features: {list(X.columns)}")
print(f"Feature shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts()}")

# Optionally export key datasets to CSV for exact diffing
pos_df.to_csv("prototype_pos_df.csv", index=False)
train_df.to_csv("prototype_train_df.csv", index=False)

import json
metrics_dict = {
    "classification_report": classification_report(y_test, preds, output_dict=True),
    "auc": roc_auc_score(y_test, probs)
}
with open("prototype_metrics.json", "w") as f:
    json.dump(metrics_dict, f, indent=2)


=== POSITIVE PAIRS DATA ===
Columns: ['registration_form_id', 'first_name_att', 'last_name_att', 'dob_norm_att', 'zip_norm_att', 'first_name_vf', 'last_name_vf', 'dob_norm_vf', 'zip_norm_vf', 'is_match']
Shape: (3081, 10)
is_match
1    3081
Name: count, dtype: int64

=== TRAINING DATA ===
Shape: (15405, 14)
is_match
0    12324
1     3081
Name: count, dtype: int64

=== FEATURES and TARGET ===
Features: ['fn_jw', 'ln_jw', 'dob_exact', 'zip_exact']
Feature shape: (15405, 4)
Target distribution:
is_match
0    12324
1     3081
Name: count, dtype: int64
